In [ ]:
!pip install --user tensorflow
!pip install --user keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding, CuDNNGRU
from tensorflow.python.keras.optimizer_v2 import adam as adam_v2
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/

/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
!ls

'A TALE OF TWO CITIES.txt'
 movie.csv
 NlpFinal.ipynb
 NlpVize
'Topic models with Gensim.ipynb adlı not defterinin kopyası'
 word2vec.ipynb
 word2vec.model


In [ ]:
dataset = pd.read_csv('movie.csv')

In [ ]:
dataset

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [ ]:
target = dataset['label'].values.tolist()
data = dataset['text'].values.tolist()

In [ ]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [ ]:
x_train[500]

"Just because an event really happened doesn't mean that it will make a good screenplay/ movie. The Cat's Meow, by Peter Bogdanovich claims to be based on actual events which happened on a cruise hosted by William Randolph Hurst. The writer paid more attention to creating a bizarre cast of characters than taking time to create a story for the bizarre characters to inhabit. The key moments of the story seem implausible; for example, when Hurst accidentally shoots the producer, believing him to be Chaplin. Basing a key element of a story on someone wearing the wrong hat is trite and contrived. The story attempts to be a dark comedy, but The Cat's Meow misses an important piece of this equation, comedy. There is also a lack of empathy for any of the characters. It hardly matters who is shot, who is killed, who is guilty and who is innocent. There is not a strong character to cheer for. As a result the conflicts are difficult to care about and the eventual outcome is incidental."

In [ ]:
x_train[800]

"Contrary to most of the comments in this section, I have to say this film just barely escapes the definition 'rubbish!'. The only readers who seem to be aware of what a catalogue of clichés it is are those who, like me, live in the north-east and know at first hand what the area is like. I am totally sick of films that are supposed to be 'realistic', yet portray the working class (of wherever) as stupid, criminal low-lifes, but then excuse them because of their social background. And funny? I smiled briefly twice and laughed once, but that was at the incongruity of two boy actors with Sunderland accents supposedly trying to hide their Newcastle accents from Sunderland football fans! There was only one likable character in the whole film, and that was the senile grandfather played quietly but very competently by Roy Hudd -- the only non-Geordie in the cast.As for the writing -- well, I just wonder what university sociology department the author studied at. He certainly had no ear for t

In [ ]:
y_train[800]

0

In [ ]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
tokenizer.fit_on_texts(data)

In [ ]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'in': 7,
 'it': 8,
 'i': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'movie': 15,
 'with': 16,
 'but': 17,
 'film': 18,
 'on': 19,
 'you': 20,
 'not': 21,
 'are': 22,
 'his': 23,
 'have': 24,
 'be': 25,
 'one': 26,
 'he': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'an': 31,
 'they': 32,
 'so': 33,
 'who': 34,
 'from': 35,
 'like': 36,
 'or': 37,
 'just': 38,
 'her': 39,
 'about': 40,
 'out': 41,
 'if': 42,
 "it's": 43,
 'has': 44,
 'there': 45,
 'some': 46,
 'what': 47,
 'good': 48,
 'more': 49,
 'when': 50,
 'very': 51,
 'up': 52,
 'no': 53,
 'time': 54,
 'even': 55,
 'my': 56,
 'would': 57,
 'she': 58,
 'which': 59,
 'only': 60,
 'really': 61,
 'see': 62,
 'story': 63,
 'their': 64,
 'had': 65,
 'can': 66,
 'me': 67,
 'were': 68,
 'well': 69,
 'much': 70,
 'than': 71,
 'we': 72,
 'been': 73,
 'get': 74,
 'bad': 75,
 'will': 76,
 'also': 77,
 'do': 78,
 'other': 79,
 'great': 80,
 'into': 81,
 'people': 82,

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

In [ ]:
x_train[800]

"Contrary to most of the comments in this section, I have to say this film just barely escapes the definition 'rubbish!'. The only readers who seem to be aware of what a catalogue of clichés it is are those who, like me, live in the north-east and know at first hand what the area is like. I am totally sick of films that are supposed to be 'realistic', yet portray the working class (of wherever) as stupid, criminal low-lifes, but then excuse them because of their social background. And funny? I smiled briefly twice and laughed once, but that was at the incongruity of two boy actors with Sunderland accents supposedly trying to hide their Newcastle accents from Sunderland football fans! There was only one likable character in the whole film, and that was the senile grandfather played quietly but very competently by Roy Hudd -- the only non-Geordie in the cast.As for the writing -- well, I just wonder what university sociology department the author studied at. He certainly had no ear for t

In [ ]:
print(x_train_tokens[800])

[3735, 5, 87, 4, 1, 801, 7, 10, 2460, 9, 24, 5, 130, 10, 18, 38, 1174, 3034, 1, 4945, 864, 1, 60, 6070, 34, 303, 5, 25, 1854, 4, 47, 3, 4, 1692, 8, 6, 22, 142, 34, 36, 67, 411, 7, 1, 2187, 2476, 2, 118, 29, 86, 523, 47, 1, 1635, 6, 36, 9, 236, 460, 1157, 4, 103, 11, 22, 440, 5, 25, 243, 2056, 1, 765, 716, 4, 7187, 13, 359, 1831, 351, 17, 90, 1378, 89, 83, 4, 64, 1128, 955, 2, 153, 9, 3031, 1472, 2, 1437, 280, 17, 11, 12, 29, 1, 4, 104, 419, 149, 16, 2530, 1539, 263, 5, 2485, 64, 2530, 35, 2030, 454, 45, 12, 60, 26, 1496, 107, 7, 1, 223, 18, 2, 11, 12, 1, 3573, 251, 5312, 17, 51, 9559, 30, 2681, 1, 60, 687, 7, 1, 176, 13, 14, 1, 485, 69, 9, 38, 581, 47, 3675, 2447, 1, 2212, 6879, 29, 27, 418, 65, 53, 4591, 14, 1, 693, 1214, 2, 9, 95, 24, 110, 545, 1, 2688, 2, 21, 60, 236, 9, 3, 2098, 144, 3, 883, 406, 19, 1, 693, 88, 446, 124, 54, 2, 19, 10, 1169, 8, 210, 153, 8, 210, 789, 8, 210, 425, 2, 8, 6]


In [ ]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [ ]:
np.mean(num_tokens)

217.364725

In [ ]:
np.max(num_tokens)

2122

In [ ]:
np.argmax(num_tokens)

34167

In [ ]:
x_test[2167]

'There\'s a sign on The Lost Highway that says:*MAJOR SPOILERS AHEAD*(but you already knew that, didn\'t you?)Since there\'s a great deal of people that apparently did not get the point of this movie, I\'d like to contribute my interpretation of why the plot makes perfect sense. As others have pointed out, one single viewing of this movie is not sufficient. If you have the DVD of MD, you can "cheat" by looking at David Lynch\'s "Top 10 Hints to Unlocking MD" (but only upon second or third viewing, please.) ;)First of all, Mulholland Drive is downright brilliant. A masterpiece. This is the kind of movie that refuse to leave your head. Not often are the comments on the DVDs very accurate, but Vogue\'s "It gets inside your head and stays there" really hit the mark.David Lynch deserves praise for creating a movie that not only has a beautifully stylish look to it - cinematography-wise, has great acting (esp. Naomi Watts), a haunting soundtrack by Badalamenti, and a very dream-like quality 

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

533

In [ ]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.945225

In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [ ]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [ ]:
x_train_pad.shape

(32000, 533)

In [ ]:
x_test_pad.shape

(8000, 533)

In [ ]:
np.array(x_train_tokens[800])

array([3735,    5,   87,    4,    1,  801,    7,   10, 2460,    9,   24,
          5,  130,   10,   18,   38, 1174, 3034,    1, 4945,  864,    1,
         60, 6070,   34,  303,    5,   25, 1854,    4,   47,    3,    4,
       1692,    8,    6,   22,  142,   34,   36,   67,  411,    7,    1,
       2187, 2476,    2,  118,   29,   86,  523,   47,    1, 1635,    6,
         36,    9,  236,  460, 1157,    4,  103,   11,   22,  440,    5,
         25,  243, 2056,    1,  765,  716,    4, 7187,   13,  359, 1831,
        351,   17,   90, 1378,   89,   83,    4,   64, 1128,  955,    2,
        153,    9, 3031, 1472,    2, 1437,  280,   17,   11,   12,   29,
          1,    4,  104,  419,  149,   16, 2530, 1539,  263,    5, 2485,
         64, 2530,   35, 2030,  454,   45,   12,   60,   26, 1496,  107,
          7,    1,  223,   18,    2,   11,   12,    1, 3573,  251, 5312,
         17,   51, 9559,   30, 2681,    1,   60,  687,    7,    1,  176,
         13,   14,    1,  485,   69,    9,   38,  5

In [ ]:
x_train_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text0 = ' '.join(words)
    return text0

In [ ]:
x_train[800]

"Contrary to most of the comments in this section, I have to say this film just barely escapes the definition 'rubbish!'. The only readers who seem to be aware of what a catalogue of clichés it is are those who, like me, live in the north-east and know at first hand what the area is like. I am totally sick of films that are supposed to be 'realistic', yet portray the working class (of wherever) as stupid, criminal low-lifes, but then excuse them because of their social background. And funny? I smiled briefly twice and laughed once, but that was at the incongruity of two boy actors with Sunderland accents supposedly trying to hide their Newcastle accents from Sunderland football fans! There was only one likable character in the whole film, and that was the senile grandfather played quietly but very competently by Roy Hudd -- the only non-Geordie in the cast.As for the writing -- well, I just wonder what university sociology department the author studied at. He certainly had no ear for t

In [ ]:
tokens_to_string(x_train_tokens[800])

"contrary to most of the comments in this section i have to say this film just barely escapes the definition ' the only readers who seem to be aware of what a of clichés it is are those who like me live in the north east and know at first hand what the area is like i am totally sick of films that are supposed to be yet portray the working class of wherever as stupid criminal low but then excuse them because of their social background and funny i briefly twice and laughed once but that was at the of two boy actors with accents supposedly trying to hide their accents from football fans there was only one likable character in the whole film and that was the grandfather played quietly but very competently by roy the only non in the cast as for the writing well i just wonder what university department the author studied at he certainly had no ear for the local accent and i too have never heard the expression and not only am i a native i'm a who's written on the local don't waste your time a

In [ ]:
model = Sequential()

In [ ]:
embedding_size = 50

In [ ]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [ ]:
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 533, 50)           500000    
_________________________________________________________________
gru_6 (GRU)                  (None, 533, 16)           3216      
_________________________________________________________________
gru_7 (GRU)                  (None, 533, 8)            600       
_________________________________________________________________
gru_8 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 503,977
Trainable params: 503,977
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train_pad = np.array(x_train_pad)
y_train = np.array(y_train)
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

Epoch 1/5
125/125 [==============================] - 142s 1s/step - loss: 0.5117 - accuracy: 0.7526
Epoch 2/5
125/125 [==============================] - 137s 1s/step - loss: 0.3305 - accuracy: 0.8753
Epoch 3/5
125/125 [==============================] - 137s 1s/step - loss: 0.2707 - accuracy: 0.9024
Epoch 4/5
125/125 [==============================] - 137s 1s/step - loss: 0.2356 - accuracy: 0.9168
Epoch 5/5
125/125 [==============================] - 135s 1s/step - loss: 0.2105 - accuracy: 0.9260


In [ ]:
x_test_pad = np.array(x_test_pad)
y_test = np.array(y_test)
result = model.evaluate(x_test_pad, y_test)

250/250 [==============================] - 55s 209ms/step - loss: 0.2913 - accuracy: 0.8840


In [ ]:
result[1]

0.8840000033378601

In [ ]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

In [ ]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [ ]:
cls_true = np.array(y_test[0:1000])

In [ ]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [ ]:
len(incorrect)

127

In [ ]:
idx = incorrect[0]
idx

0

In [ ]:
text0 = x_test[idx]
text0

"This movie was a major bait and switch. I rented it because of Rebecca St. James, a popular Christian singer. I have met her and wondered what she would be doing in a UFO movie. Well.......I think that she starred in this movie to help out a friend, or a friend of a friend. My first clue that this movie wasn't what it was supposed to be was when I witnessed the special effects of the UFO encounters. Cheesy! As the movie progressed, I noticed how plastic the actors were. It was funny how almost everyone in the movie wore solid colors. (There are a few exceptions).Rebecca was verrryyy disappointing. She is always found in the house and doesn't show the realistic facial expressions of one whose husband has return to the fold. Doesn't she ever leave the house? I had to turn off the movie several times in order to finish it. I hope that Rebecca doesn't believe the message of this movie - believe in what we believe or suffer and go to hell. Jesus spread a message of love and hope. His messa

In [ ]:
y_pred[idx]

0.9440327

In [ ]:
cls_true[idx]

0

In [ ]:
text1 = " A thrilling masterpiece worth the long run time even better in IMAX! Important to note Rip Lance Reddick he was amazing in his career!"
text2 = "Hands down the best action movie franchise ever."
text3 = "John Wick: Chapter 4 offered me 169 minutes of pure entertainment with an exciting cast of characters and action sequences that'll impress anyone, a perfect action film"
text4 = "Bad acting, bad casting, bad adaptation. Do not watch this if you enjoyed the book. From start to finish it's bad, there's no redeeming qualities to this film! Parker looks about 50. The twins, yes fraternal but one looks 10 years older. One of the reasons Travis was interested in Abby was her not being interested in him, yet she seems to pine after him. Their relationship is not believable. You don't get the idea that Travis sleeps around with everyone and makes a change for her. I cannot believe how bad this film is. The actors are awful, not casted well at all and the acting is atrocious. Disappointed from start to finish."
text5 = "This movie was nothing like the book, except for the character's names. The acting was atrocious and there was barely any chemistry...I came for the steam, but there is no water in my iron, so everything came out forced. I did not believe him as a fighter, so every scene he fought in made me laugh...his one liners made him seem more Spiderman than a brawler. Her gambling scene seemed more juvenile than something that would happen in real life. There was some funny moments but at one point I was wondering if I was laughing because it was funny or because it was sad."
text6 = "I'm a huge fan of the book. I was so excited when the movie was announced. Man, did the screenwriter even read the book? This is the biggest jumbled mess of crap! The dialogue is cringe worthy. The movie felt so rushed...I couldn't root for them as a couple. They left out so many potential scenes. The acting is over the top and cheesy. The actress playing Abby was so bad! I was hesitant of Dylan playing the part at first...but I will say that I was pleasantly surprised with him. He handled the fight scenes perfectly! As for the actors playing his brothers? No on ALL counts! What a way to ruin a book! When they say the book was better...they are right! Don't waste your time!"
text7 = "It all leads back to where we once started off as all great trilogies have indicated from the past. But GOTG surpasses expectations with what is nothing short of phenomenal performances by the cast and truly well written and does not shy away from what could only be understood from what requires a sense of maturity to understand. Rockets back story alone is on occasion dark and incredibly sad it had me in tears at a few instances during the film. Honestly I did not expect this, James Gunn yet again provides an absolutely beautiful piece of cinematography and I have to say I really hope in the future they dive back into the story of the colony . With Adam warlock finally in the MCU it's most likely there will be some cameos. But it does deliver in the traditional guardians humor but this film is meant about more for the mature audience of marvel. Trust me this is a great film."
text8 = "This. This is what I've wanted. Yeah some of the jokes are a bit too silly and the tone is a bit confusing at times. But the end result for me is probably the most heartfelt emotional reaction from me to a comic book movie since Endgame. It's not a perfect film, I am conflicted with what the did with Adam Warlock. And the main villain is a little hammy. But the personal relationship between the guardians and where they take them is dark, heartbreaking, and also beautiful. This is also the best performance from Chris Pratt probably his entire time playing this character. But the person who steals the show is Rocket. I don't know how but they need to change the rules degree voice acting performances can be nominated for Oscars because it was incredible. This is the most emotionally satisfying experience I've had with the MCU in a very long time. And every critic who is saying that the movie is too schmaltzy or sappy... doesn't understand how much these characters not only mean to people but also James Gunn. This isn't just a final chapter of a story, it's a goodbye to a set of characters and films that changed the landscape of not only the tone and vibe of superhero movies but also changed the fact that weird unknown characters who have strange stories can be highlighted in these films."
text9 = "I think for what this movie sets out to do, Super Mario Bros. Movie pretty much ticks the boxes of an entertaining, reference fan-service heavy movie. A lot of the jokes are basic sure, but it's just impressive that there is some competency to a Video Game Movie. Sometimes simple and straightforward is the best way to go. Not high art or anything, but it ticks off almost everything for what Mario should be at least. Voice acting wasn't as bad as I thought, and Jack Black was the highlight as expected. Yeah, not gonna win any big awards but judge it for what it is, and yeah by Video Game Movie standards it's one of the best ones easily if you ask me."
text10 = "The story was simple, but worked wonders with the excellent characters that perfectly reflected their game counterparts. They even had unique aspects given to them that helped them stand out in all the right ways. The action was phenomenal with great depth and detail that shows how much work was put into this. There were so many nods and references to other classic Nintendo properties that you'd want to go back to find them all. And Charles Martinet is in this with a small but great role. But with all that said, it does mess up its flow with some of the music. Definitely not the original score though, since it mixes the classic music with an epic and emotional tone. But the copyrighted music Illumination probably payed a third of their budget for felt out of place in this film and took me out of the experience a couple times. Other than that, it's an amazing that I'll gladly watch again."
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [ ]:
tokens = tokenizer.texts_to_sequences(texts)

In [ ]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

(10, 533)

In [ ]:
model.predict(tokens_pad)

array([[0.89164174],
       [0.50121   ],
       [0.6166402 ],
       [0.03234053],
       [0.07595197],
       [0.0389641 ],
       [0.962625  ],
       [0.86469615],
       [0.36488223],
       [0.96354073]], dtype=float32)